In [1]:
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf62480413a152b510459cabf5e54bcfb4a7a9')
#client = ors.Client(key='5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf')

In [2]:
import re
import pandas as pd
import math

import numpy as np

from operator import itemgetter
from heapq import nsmallest

from itertools import chain

import time

In [3]:
df_property = pd.read_csv('../data/curated/property.csv')

property_coor = []
for index, row in df_property.iterrows():
    lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
    long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
    property_coor.append((long,lat))

print(len(property_coor))

df_train = pd.read_csv('../data/curated/train_coord.csv')

train_coor = []
for index, row in df_train.iterrows():
    lat = row['0']
    long = row['1']
    train_coor.append((long,lat))
    
print(len(train_coor))

15222
332


In [4]:
import geopy.distance

In [ ]:
# find the closest 3 train stations for each property based on distance
# api has accessed limits so that it is difficult to track the route between each train station and each property


def get_train(index):

    # calculate distance between the property and each train station
    distances = [geopy.distance.geodesic((property_coor[index][1], property_coor[index][0]), (train_coor[j][1], train_coor[j][0])).km for j in range(len(train_coor))]

    # find the 3 nearest train stations
    index = list(range(len(train_coor)))
    pairs = zip(index, distances)
    result = nsmallest(3, pairs, key=itemgetter(1))
    index_list = [i[0] for i in result]

    # store the name and the coordinates of the 3 nearest train stations
    name_list = [df_train['School_Name'][i] for i in index_list]
    coor_list = [train_coor[i] for i in index_list]
    
    result = []
    result.append(name_list)
    result.append(coor_list)

    return result

# for each property, get corresponding closest train stations
name_list = []
coor_list = []

for i in range(len(property_coor)):
    result = get_train(i)
    name_list.append(result[0])
    coor_list.append(result[1])

print(len(name_list))

In [ ]:
def get_train_ors(iter):


    flatten_list = list(chain.from_iterable(coor_list[iter*34:iter*34+34]))
    combine_coor = property_coor[iter*34:iter*34+34] + flatten_list

    if iter == len(df_property)//34:
        count = len(df_property)-iter*34
    else:
        count = 34

    matrix = client.distance_matrix(
        locations=combine_coor,
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
        destinations=list(range(count,len(combine_coor))),
        sources=list(range(count)),
    )

    duration_list = []
    distance_list = []
    train_name_list = []
    train_coor_list = []


    for i in range(count):

        min_index = np.argmin(matrix['durations'][i][i*3:i*3+3])
        min_index_flat = min_index+i*3
        duration_list.append(matrix['durations'][i][min_index_flat])
        distance_list.append(matrix['distances'][i][min_index_flat])

        train_name_list.append(name_list[iter*34+i][min_index])
        train_coor_list.append(coor_list[iter*34+i][min_index])


    #print("Durations in secs: {}\n".format(matrix['durations']))
    #print("Distances in m: {}".format(matrix['distances']))

    # we only want to call the whole function once at each main iteration
    result_list = []
    result_list.append(duration_list)
    result_list.append(distance_list)
    result_list.append(train_name_list)
    result_list.append(train_coor_list)

    return result_list

# A total of 3500 routes is allowed for each call of api, so that we can pass 34 groups of property 
# with corresponding 3 closest schools at each time

duration_list = []
distance_list = []
train_name_list = []
train_coor_list = []

num_call = len(df_property)//34 + 1
for i in range(34,35):
    result = get_train_ors(i)
    '''
    if i%30 == 0:
        time.sleep(60)
    '''
    duration_list.append(result[0])
    distance_list.append(result[1])
    train_name_list.append(result[2])
    train_coor_list.append(result[3])

print(duration_list)
print(distance_list)
print(train_name_list)
print(train_coor_list)

In [ ]:
flatten_duration_list = list(chain.from_iterable(duration_list))
flatten_distance_list = list(chain.from_iterable(distance_list))
flatten_train_name_list = list(chain.from_iterable(train_name_list))
flatten_train_coor_list = list(chain.from_iterable(train_coor_list))

print(len(flatten_distance_list))